# Text Translation and Sentiment Analysis using Transformers

## Project Overview:

The objective of this project is to analyze the sentiment of movie reviews in three different languages - English, French, and Spanish. We have been given 30 movies, 10 in each language, along with their reviews and synopses in separate CSV files named `movie_reviews_eng.csv`, `movie_reviews_fr.csv`, and `movie_reviews_sp.csv`.

- The first step of this project is to convert the French and Spanish reviews and synopses into English. This will allow us to analyze the sentiment of all reviews in the same language. We will be using pre-trained transformers from HuggingFace to achieve this task.

- Once the translations are complete, we will create a single dataframe that contains all the movies along with their reviews, synopses, and year of release in all three languages. This dataframe will be used to perform sentiment analysis on the reviews of each movie.

- Finally, we will use pretrained transformers from HuggingFace to analyze the sentiment of each review. The sentiment analysis results will be added to the dataframe. The final dataframe will have 30 rows


The output of the project will be a CSV file with a header row that includes column names such as **Title**, **Year**, **Synopsis**, **Review**, **Review Sentiment**, and **Original Language**. The **Original Language** column will indicate the language of the review and synopsis (*en/fr/sp*) before translation. The dataframe will consist of 30 rows, with each row corresponding to a movie.

In [1]:
# imports
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer
from transformers import pipeline

2024-10-12 03:57:26.475558: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-12 03:57:27.077447: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-12 03:57:28.497940: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### Get data from `.csv` files and then preprocess data

In [21]:
# use the `pd.read_csv()` function to read the movie_review_*.csv files into 3 separate pandas dataframes

# Note: All the dataframes would have different column names. For testing purposes
# you should have the following column names/headers -> [Title, Year, Synopsis, Review]

def preprocess_data() -> pd.DataFrame:
    """
    Reads movie data from .csv files, map column names, add the "Original Language" column,
    and finally concatenate in one resultant dataframe called "df".
    """
    df_eng = pd.read_csv("data/movie_reviews_eng.csv")
    df_spa = pd.read_csv("data/movie_reviews_sp.csv")
    df_fra = pd.read_csv("data/movie_reviews_fr.csv")

    mapping = {
        'df_eng': {
            'Movie / TV Series': 'Title',
            'Year': 'Year',
            'Synopsis': 'Synopsis',
            'Review': 'Review'
        },
        'df_spa': {
            'Título': 'Title',
            'Año': 'Year',
            'Sinopsis': 'Synopsis',
            'Críticas': 'Review'
        },
        'df_fra': {
            'Titre': 'Title',
            'Année': 'Year',
            'Synopsis': 'Synopsis',
            'Critiques': 'Review'
        }
    }

    df_eng = df_eng.rename(columns=mapping['df_eng'])
    df_spa = df_spa.rename(columns=mapping['df_spa'])
    df_fra = df_fra.rename(columns=mapping['df_fra'])

    df_eng['Original Language'] = 'en'
    df_spa['Original Language'] = 'es'
    df_fra['Original Language'] = 'fr'

    df = pd.concat([df_eng, df_spa, df_fra], ignore_index=True)

    return df

df = preprocess_data()
df.head()

,Title,Year,Synopsis,Review,Original Language
0,The Shawshank Redemption,1994,"Andy Dufresne (Tim Robbins), a successful bank...","""The Shawshank Redemption is an inspiring tale...",en
1,The Dark Knight,2008,Batman (Christian Bale) teams up with District...,"""The Dark Knight is a thrilling and intense su...",en
2,Forrest Gump,1994,Forrest Gump (Tom Hanks) is a simple man with ...,"""Forrest Gump is a heartwarming and inspiratio...",en
3,The Godfather,1972,Don Vito Corleone (Marlon Brando) is the head ...,"""The Godfather is a classic movie that stands ...",en
4,Inception,2010,Dom Cobb (Leonardo DiCaprio) is a skilled thie...,"""Inception is a mind-bending and visually stun...",en


In [22]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language
22,Amélie,2001,Cette comédie romantique raconte l'histoire d'...,"""Amélie est un film absolument charmant qui vo...",fr
2,Forrest Gump,1994,Forrest Gump (Tom Hanks) is a simple man with ...,"""Forrest Gump is a heartwarming and inspiratio...",en
18,Torrente: El brazo tonto de la ley,1998,"En esta comedia española, un policía corrupto ...","""Torrente es una película vulgar y ofensiva qu...",es
0,The Shawshank Redemption,1994,"Andy Dufresne (Tim Robbins), a successful bank...","""The Shawshank Redemption is an inspiring tale...",en
10,Roma,2018,Cleo (Yalitza Aparicio) es una joven empleada ...,"""Roma es una película hermosa y conmovedora qu...",es
15,Águila Roja,(2009-2016),Esta serie de televisión española sigue las av...,"""Águila Roja es una serie aburrida y poco inte...",es
17,El Bar,2017,Un grupo de personas quedan atrapadas en un ba...,"""El Bar es una película ridícula y sin sentido...",es
7,The Nice Guys,2016,"In 1970s Los Angeles, a private eye (Ryan Gosl...","""The Nice Guys tries too hard to be funny, and...",en
14,Amores perros,2000,Tres historias se entrelazan en esta película ...,"""Amores perros es una película intensa y conmo...",es
8,Solo: A Star Wars Story,2018,A young Han Solo (Alden Ehrenreich) joins a gr...,"""Dull and pointless, with none of the magic of...",en


### Text translation

Translate the **Review** and **Synopsis** column values to English.

In [34]:
# load translation models and tokenizers
fr_en_model_name = "Helsinki-NLP/opus-mt-fr-en"
es_en_model_name = "Helsinki-NLP/opus-mt-es-en"
fr_en_model = MarianMTModel.from_pretrained(fr_en_model_name)
es_en_model = MarianMTModel.from_pretrained(es_en_model_name)
fr_en_tokenizer = MarianTokenizer.from_pretrained(fr_en_model_name)
es_en_tokenizer = MarianTokenizer.from_pretrained(es_en_model_name)

# TODO 3: Complete the function below
def translate(text: str, model, tokenizer) -> str:
    """
    function to translate a text using a model and tokenizer
    """
    # encode the text using the tokenizer
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    
    # generate the translation using the model
    outputs = model.generate(**inputs)

    # decode the generated output and return the translated text
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [40]:
# filter reviews in French and translate to English
fr_reviews = df[df['Original Language'] == 'fr']['Review']
fr_reviews_en = fr_reviews.apply(
        lambda x: translate(x, fr_en_model, fr_en_tokenizer)
    )
print('fr_reviews_en')
print(fr_reviews_en)

# filter synopsis in French and translate to English
fr_synopsis = df[df['Original Language'] == 'fr']['Synopsis']
fr_synopsis_en = fr_synopsis.apply(
    lambda x: translate(x, fr_en_model, fr_en_tokenizer)
    )
print('fr_synopsis_en')
print(fr_synopsis_en)

# filter reviews in Spanish and translate to English
es_reviews = df[df['Original Language'] == 'es']['Review']
es_reviews_en = es_reviews.apply(
    lambda x: translate(x, es_en_model, es_en_tokenizer)
    )

print('es_reviews_en')
print(es_reviews_en)

# filter synopsis in Spanish and translate to English
es_synopsis = df[df['Original Language'] == 'es']['Synopsis']
es_synopsis_en = es_synopsis.apply(
    lambda x: translate(x, es_en_model, es_en_tokenizer)
)
print('es_synopsis_en')
print(es_synopsis_en)

# update dataframe with translated text
# add the translated reviews and synopsis - you can overwrite the existing data
df.loc[df['Original Language'] == 'fr', 'Review'] = fr_reviews_en
df.loc[df['Original Language'] == 'fr', 'Synopsis'] = fr_synopsis_en
df.loc[df['Original Language'] == 'es', 'Review'] = es_reviews_en
df.loc[df['Original Language'] == 'es', 'Synopsis'] = es_synopsis_en

fr_reviews_en
20    "The Land is an absolutely beautiful film with...
21    "Untouchables is an incredibly touching film w...
22    "Amélie is an absolutely charming film that wi...
23    "The Choristes are a beautiful film that will ...
24    "The Fabulous Destiny of Amélie Poulain is an ...
25    "I didn't like this movie at all. The concept ...
26    "I can't believe I've wasted time watching thi...
27    "This film is a complete disappointment. The j...
28    "The film is a total waste of time. The jokes ...
29    "This film is a complete mess. The characters ...
Name: Review, dtype: object
fr_synopsis_en
20    This musical tells the story of a budding actr...
21    This film tells the story of the unlikely frie...
22    This romantic comedy tells the story of Amélie...
23    This film tells the story of a music teacher w...
24    This romantic comedy tells the story of Amélie...
25    The film follows the story of a group of rich ...
26    Two incompetent office workers find thems

In [41]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language
26,La Tour Montparnasse Infernale,2001,Two incompetent office workers find themselves...,"""I can't believe I've wasted time watching thi...",fr
29,Babylon A.D.,2008,"In the distant future, a mercenary has to esco...","""This film is a complete mess. The characters ...",fr
8,Solo: A Star Wars Story,2018,A young Han Solo (Alden Ehrenreich) joins a gr...,"""Dull and pointless, with none of the magic of...",en
24,Le Fabuleux Destin d'Amélie Poulain,2001,This romantic comedy tells the story of Amélie...,"""The Fabulous Destiny of Amélie Poulain is an ...",fr
4,Inception,2010,Dom Cobb (Leonardo DiCaprio) is a skilled thie...,"""Inception is a mind-bending and visually stun...",en
22,Amélie,2001,This romantic comedy tells the story of Amélie...,"""Amélie is an absolutely charming film that wi...",fr
18,Torrente: El brazo tonto de la ley,1998,"In this Spanish comedy, a corrupt cop (played ...","""Torrente is a vulgar and offensive film that ...",es
9,The Island,2005,In a future where people are cloned for organ ...,"""The Island is a bland and forgettable sci-fi ...",en
10,Roma,2018,Cleo (Yalitza Aparicio) is a young domestic wo...,"""Rome is a beautiful and moving film that pays...",es
20,La La Land,2016,This musical tells the story of a budding actr...,"""The Land is an absolutely beautiful film with...",fr


### Sentiment Analysis

Use HuggingFace pretrained model for sentiment analysis of the reviews. Store the sentiment result **Positive** or **Negative** in a new column titled **Sentiment** in the dataframe.

In [47]:
# TODO 5: Update the code below
# load sentiment analysis model
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
sentiment_classifier = pipeline("sentiment-analysis", model=model_name)

# TODO 6: Complete the function below
def analyze_sentiment(text, classifier):
    """
    function to perform sentiment analysis on a text using a model
    """
    result = classifier(text)
    return result[0]['label'], result[0]['score']

In [48]:
# perform sentiment analysis on reviews and store results in new column
df['Sentiment'], df['Sentiment_Score'] = zip(*df['Review'].apply(lambda x: analyze_sentiment(x, sentiment_classifier)))

In [50]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language,Sentiment,Sentiment_Score
11,La Casa de Papel,(2017-2021),This Spanish television series follows a group...,"""The Paper House is an exciting and addictive ...",es,POSITIVE,0.999890
26,La Tour Montparnasse Infernale,2001,Two incompetent office workers find themselves...,"""I can't believe I've wasted time watching thi...",fr,NEGATIVE,0.999815
28,Les Visiteurs en Amérique,2000,In this continuation of the French comedy The ...,"""The film is a total waste of time. The jokes ...",fr,NEGATIVE,0.999819
14,Amores perros,2000,Three stories intertwine in this Mexican film:...,"""Amores dogs is an intense and moving film tha...",es,POSITIVE,0.999887
12,Y tu mamá también,2001,Two teenage friends (Gael García Bernal and Di...,"""And your mom is also a movie that stays with ...",es,POSITIVE,0.999874
29,Babylon A.D.,2008,"In the distant future, a mercenary has to esco...","""This film is a complete mess. The characters ...",fr,NEGATIVE,0.999814
21,Intouchables,2011,This film tells the story of the unlikely frie...,"""Untouchables is an incredibly touching film w...",fr,POSITIVE,0.999881
27,Astérix aux Jeux Olympiques,2008,In this film adaptation of the popular comic s...,"""This film is a complete disappointment. The j...",fr,NEGATIVE,0.999798
18,Torrente: El brazo tonto de la ley,1998,"In this Spanish comedy, a corrupt cop (played ...","""Torrente is a vulgar and offensive film that ...",es,NEGATIVE,0.999754
17,El Bar,2017,A group of people are trapped in a bar after M...,"""The Bar is a ridiculous and meaningless film ...",es,NEGATIVE,0.999808


In [51]:
# export the results to a .csv file
df.to_csv("./reviews_with_sentiment.csv", index=False)

OSError: Cannot save file into a non-existent directory: 'result'